# Referenced Mosaic


# Setup


## Imports


In [ ]:
import os
import shutil

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

In [ ]:
from night_horizons import utils
from night_horizons.mapmake import MosaicMaker

## Settings


In [ ]:
local_options = {
    'io_manager': {
        'output_dir': 'mosaics/referenced',
        'file_exists': 'overwrite',
    },
    'overwrite': True,
    'random_state': 16849,
}

In [ ]:
# This is a pre-built factory to make the objects needed for mosaicking,
# updated to use the config.
mosaic_maker = MosaicMaker(
    config_filepath='./config.yml',
    local_options=local_options,
)

## Parse Settings


In [ ]:
io_manager = mosaic_maker.container.get_service('io_manager')
referenced_fps = io_manager.input_filepaths['referenced_images']

In [ ]:
assert len(referenced_fps) > 0

In [ ]:
test_filenames = {
    'mosaic': 'mosaic.tiff',
    'settings': 'settings.yaml', 
    'checkpoint': os.path.join(
        'checkpoints',
        f"mosaic_i{100:06d}.tiff"
    ),
}
check_fps = {
    key: os.path.join(io_manager.output_dir, value)
    for key, value in test_filenames.items()
}

# Extract/Transform/Load


In [ ]:
preprocessor = mosaic_maker.container.get_service('preprocessor')
X = preprocessor.fit_transform(
    referenced_fps,
)

# Build Mosaic


In [ ]:
mosaicker = mosaic_maker.container.get_service('mosaicker')
X_out = mosaicker.fit_transform(X)

# Evaluate


## Check Output Exists


In [ ]:
for key, fp in check_fps.items():
    if len(referenced_fps) < 100 and key == 'checkpoint':
        continue
    assert os.path.isfile(fp), f'Did not find file {key}: {fp}'

## Individual Inspection


In [ ]:
ind = mosaic_maker.container.config['random_state'].choice(X.index)
row = X_out.loc[ind]

In [ ]:
original_img = utils.load_image(row['filepath'])
dataset = io_manager.open_dataset()
mosaic_img = mosaicker.processor.get_image_from_dataset(
    dataset, row['x_off'], row['y_off'], row['x_size'], row['y_size'])

In [ ]:
subplot_mosaic = [['original', 'mosaic']]
n_rows = len(subplot_mosaic)
n_cols = len(subplot_mosaic[0])

fig = plt.figure(figsize=(10*n_cols, 10*n_rows))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['original']
ax.imshow(original_img)

ax = ax_dict['mosaic']
ax.imshow(mosaic_img)

## Overall Scores


In [ ]:
r_median = mosaicker.score(X)

In [ ]:
sns.histplot(mosaicker.scores_)

In [ ]:
sns.scatterplot(
    x=np.arange(len(mosaicker.scores_)),
    y=mosaicker.scores_,
)

## Calculate the area


In [ ]:
full_mosaic_img = mosaicker.get_image(dataset, 0, 0, mosaicker.x_size_, mosaicker.y_size_)

In [ ]:
is_not_empty = ~np.isclose(full_mosaic_img, 0.)

In [ ]:
full_area = is_not_empty.sum() * -mosaicker.pixel_height_ * mosaicker.pixel_width_
full_area_km = full_area / (1000.)**2.
print(f'The area covered by the referenced mosaic is {full_area_km:.2g} km^2')

In [ ]:
# From google
actual_area_km = 953.

In [ ]:
# We only check if the area is large enough because we image the area around Indianapolis too
assert full_area_km > actual_area_km

## Cleanup


In [ ]:
dataset.FlushCache()
dataset = None